In [ ]:
import pickle
from random import randrange
import itertools
import random
import pandas as pd
import numpy as np
import math
from gams import *

From_depot_to_trip_time = pd.read_csv("From_depot_to_trip_time_3line")
From_trip_to_depot_time = pd.read_csv("From_trip_to_depot_time_3line")
From_depot_to_trip_energy = pd.read_csv("From_depot_to_trip_energy_3line")
From_trip_to_depot_energy = pd.read_csv("From_trip_to_depot_energy_3line")

trip_information_3line = pd.read_csv("Trip_information_3line.csv")
terminal_distance_3line = pd.read_csv("Between_terminal_distance_3line.csv")

In [ ]:
######function to get the maximum available time for charging between trips if go back to depot for charging between trips
def avail_char_time_between_trip_depot(trip_information, terminal_distance):
    avail_char_time_between_trip_depot = pd.DataFrame()
    i=0
    while i < 70:
        time_list_depot = []
        j = 0
        while j < 70:
            trip1 = trip_information.iloc[i]["Trip_no"]
            trip2 = trip_information.iloc[j]["Trip_no"]
            trip1_starttime = trip_information.iloc[i]["Start_time"]
            trip2_starttime = trip_information.iloc[j]["Start_time"]
            trip1_endtime = trip_information.iloc[i]["End_time"]
            trip2_endtime = trip_information.iloc[j]["End_time"]
            trip1_startterminal = trip_information.iloc[i]["Start_terminal"]
            trip2_startterminal = trip_information.iloc[j]["Start_terminal"]
            trip1_endterminal = trip_information.iloc[i]["End_terminal"]
            trip2_endterminal = trip_information.iloc[j]["End_terminal"]
            from_trip1_to_depot_time = From_trip_to_depot_time.iloc[i][1]
            from_depot_to_trip2_time = From_depot_to_trip_time.iloc[j][1]
            if trip1 == trip2:
                time_list_depot.append(-1000)
            else:
                if trip2_starttime < trip1_endtime + from_trip1_to_depot_time*60 + from_depot_to_trip2_time*60:
                    time_list_depot.append(-1000)
                else:
                    remain_time = (trip2_starttime - trip1_endtime)/60 - from_trip1_to_depot_time - from_depot_to_trip2_time
                    time_list_depot.append(remain_time)
            j += 1
        time_list_depot = pd.DataFrame(time_list_depot)
        time_list_depot = time_list_depot.transpose()
        avail_char_time_between_trip_depot = avail_char_time_between_trip_depot.append(time_list_depot)
        i+=1
    return avail_char_time_between_trip_depot

In [ ]:
avail_char_time_between_trip_depot.to_csv("Avail_char_time_between_trip_depot_3line.csv")
avail_char_time_between_trip_depot = avail_char_time_between_trip_depot(trip_information_3line, terminal_distance_3line)
avail_char_time_between_trip_depot.to_csv("Avail_char_time_between_trip_depot_3line.csv")
following_set = pickle.load(open('Following_set_dict_3line.pickle','rb'))

In [ ]:
#####################################
#define a function to get a following trip of a service trip
#####################################
def return_following(i,following_set):
    following_set_i = following_set[i]
    if len(following_set_i) == 0:
        return -1
    else:
        following_set_i = following_set_i[0:5]
        return(random.choice(following_set_i))

In [ ]:
#####################################
#define a function to generate a block given a first trip
#####################################
def block_generation(i,block,following_set):
    while True:
        m = return_following(i,following_set)
        if m != -1: 
            block.append(m)
            i = m
        else: 
            #print("block=")
            #print(block)
            return block

In [ ]:
####################################
#define a function to get difference of two lists
####################################
def diff(list1, list2):
    return (list(list(set(list1)-set(list2)) + list(set(list2)-set(list1))))

In [ ]:
####################################
#define a function to generate the service trip part of a chromosome
####################################
def trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip):
    while True:
        if len(rest_trips) != 0:
            fist_trip = min(rest_trips)                        #get the trip with the earlest starting time in rest of the trips
            new_block = []
            new_block.append(fist_trip)
            new_block = block_generation(fist_trip,new_block,following_set)  #generate a new block
            rest_trips = diff(rest_trips, new_block)           #get rest of the trips
            chromosome = chromosome + new_block                #add the newly generated block to the chromosome
            trip_chains.append(new_block)
            no_bus += 1
            no_trip.append(len(new_block))
            following_set = {key:[ele for ele in following_set[key] if ele not in chromosome] for key in following_set.keys()}
        else:
            #print("All trips have been assigned to blocks")
            #print("Generated trip part of chromosome =")
            #print(trip_chains)
            #print("Fleet size under this trip schedule =")
            #print(no_bus)
            #print("Number of trips for each bus =")
            #print(no_trip)
            
            return trip_chains, no_bus, no_trip

In [ ]:
#########################################
#define a function to randomly assign a battery capacity for each bus in the bus fleet
#########################################
def battery_cap_generation(fleet_size):
    bus_type = [1,2]                   #available bus type in the market
    battery_type = [150,380]            #battery capacity of each bus type
    battery_size = [0]*fleet_size      #define a list to store the battery size of each bus in the bus fleet
    battery_lower = [0]*fleet_size     #define the lower bound of battery energy in bus
    battery_upper = [0]*fleet_size     #define the upper bound of battery energy in bus
    for i in range(fleet_size):        #randomly assign a battery capacity for each bus in the bus fleet
        battery_size[i] = random.choice(battery_type)
        battery_lower[i] =  battery_size[i]*0.2
        battery_upper[i] = battery_size[i]*0.9
    return battery_size, battery_lower, battery_upper

In [ ]:
#########################################
#define a function to randomly choose whether to go back to the depot for charging between trips
#########################################
def whether_charge_generation(fleet_size, trip_chains, trips_per_bus):
    whether_charge_set = []
    i = 0
    while (i < fleet_size):
        trip_chain = trip_chains[i]                 #get the trip chain of the bus i
        no_trips_per_bus = trips_per_bus[i]         #get the number of trips for the bus i
        if no_trips_per_bus == 1:
            whether_charge_set.append([])
        else:
            b=np.random.randint(2, size=no_trips_per_bus-1)
            whether_charge_set.append(b)
        i += 1
    return whether_charge_set

In [ ]:
########################################
#define the function to check feasibility of the generated chromosome
########################################
def check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains, whether_charge_set, avail_char_time_between_trip_depot):
    i = 0
    while (i < fleet_size):
        #print("i=" +str(i))
        a = trips_per_bus[i]                        #get the number of trips for the bus i
        battery_l = battery_lower[i]                #get the battery capacity lower bound of the bus i
        battery_u = battery_upper[i]                #get the battery capacity upper bound of the bus i
        trip_chain = trip_chains[i]                 #get the trip chain of the bus i
        #print(trip_chain)
        b = trip_chain[0]                           #get the first trip in the trip chain of the bus i
        remain_energy = battery_u - From_depot_to_trip_energy.iloc[b-1][1] - trip_information_3line.iloc[b-1]["Energy_need"]   #get the remaining energy in bus i after finishing its first trip
        if remain_energy < battery_l:
            #print("The depot to first trip part of the trip chain "+str(i)+" is not feasible")
            i = 1000
            return 0
            break
        else:
            if a == 1:
                remain_energy = remain_energy - From_trip_to_depot_energy.iloc[b-1][1]
                if remain_energy < battery_l:
                    #print("The only trip to depot part of the trip chain "+str(i)+" is not feasible")
                    i = 1000
                    return 0
                    break
                #else:
                    #print("Trip chain "+str(i)+" is feasible")
            elif a > 1:
                j = 1
                while (j < a):
                    #print("j= "+str(j))
                    bb = trip_chain[j] 
                    #print("bb= " +str(bb))
                    cc = trip_chain[j-1] 
                    #print("cc= " +str(cc))
                    whether_to_charge = whether_charge_set[i][j-1]
                    if j < a-1:
                        if whether_to_charge == 0:
                            remain_energy = remain_energy - energy_need.iloc[cc-1][bb] - trip_information_3line.iloc[bb-1]["Energy_need"]
                            if remain_energy < battery_l:
                            #print("The between trips part "+str(cc)+" to "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                                i = 1000
                                return 0
                                break
                        else:
                            remain_energy = remain_energy - From_trip_to_depot_energy.iloc[cc-1][1]
                            if remain_energy < battery_l:
                                i = 1000
                                return 0
                                break
                            else:
                                remain_energy = remain_energy + 350*avail_char_time_between_trip_depot.iloc[cc-1][bb-1]
                                if remain_energy > battery_u:
                                    remain_energy = battery_u
                                else: 
                                    remain_energy = remain_energy
                            remain_energy = remain_energy - From_depot_to_trip_energy.iloc[bb-1][1] - trip_information_3line.iloc[bb-1]["Energy_need"]
                            if remain_energy < battery_l:
                                #print("The between trips part "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                                i = 1000
                                return 0
                                break
                    else:
                        if whether_to_charge == 0:
                            remain_energy = remain_energy - energy_need.iloc[cc-1][bb] - trip_information_3line.iloc[bb-1]["Energy_need"] - From_trip_to_depot_energy.iloc[bb-1][1]
                            if remain_energy < battery_l:
                            #print("The between trips part "+str(cc)+" to "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                                i = 1000
                                return 0
                                break
                        else:
                            remain_energy = remain_energy - From_trip_to_depot_energy.iloc[cc-1][1]
                            if remain_energy < battery_l:
                                i = 1000
                                return 0
                                break
                            else:
                                remain_energy = remain_energy + 350*avail_char_time_between_trip_depot.iloc[cc-1][bb-1]
                                if remain_energy > battery_u:
                                    remain_energy = battery_u
                                else: 
                                    remain_energy = remain_energy
                            remain_energy = remain_energy - From_depot_to_trip_energy.iloc[bb-1][1] - trip_information_3line.iloc[bb-1]["Energy_need"] - From_trip_to_depot_energy.iloc[bb-1][1]
                            if remain_energy < battery_l:
                                #print("The between trips part "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                                i = 1000
                                return 0
                                break
                    j += 1
        i += 1
    return 1


In [ ]:
## generate the trip part of the chromosome
rest_trips = [i for i in range(1,71)]
chromosome = []
#crosspoint_trip = []
#crosspoint_batt = []
no_bus = 0
no_trip = []
trip_chains = []
trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)

In [ ]:
#trip_chromosome=trippart_chromosome[0]
trip_chains=trippart_chromosome[0]
#crossover_point=trippart_chromosome[2]
fleet_size=trippart_chromosome[1]
trips_per_bus=trippart_chromosome[2]

In [ ]:
### generate the battery capacity for each bus
battery_cap = battery_cap_generation(fleet_size)
battery_size = battery_cap[0]
battery_lower = battery_cap[1]
battery_upper = battery_cap[2]

In [ ]:
### randomly determine whether to go back to the depot for charging between trips
whether_charge_set = whether_charge_generation(fleet_size, trip_chains, trips_per_bus)
print(whether_charge_set)

In [ ]:
### check feasibility of the generated chromosome
check = check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains, whether_charge_set, avail_char_time_between_trip_depot)
print(check)

In [ ]:
Initial_population = pickle.load(open('Initial_population_3line_only_depot_charging.pickle','rb'))

In [ ]:
##########################################
#choose the smaller one from two items
##########################################
def compare_two(num1, num2):
    if num1 >= num2:
        return num2
    else:
        return num1
    
    
##########################################
#function for implementing the single-point crossover
##########################################
def crossover(l1, l2, k):
    l_new = []
    for i in range(0, k):
        l_new.append(l1[i])
    for j in range(k, len(l2)):
        l_new.append(l2[j])
    return l_new
        

##########################################
#crossover between two chromosomes
##########################################
def cross_over(chrom1, chrom2):
    off_spring = {}
    chrom1_trip = chrom1["trip_chromosome"]
    chrom2_trip = chrom2["trip_chromosome"]
    chrom1_batt = chrom1["battery_cap"]
    chrom2_batt = chrom2["battery_cap"]
    chrom1_flze = chrom1["fleet_size"]
    chrom2_flze = chrom2["fleet_size"]
    smaller_fleet_size = compare_two(chrom1_flze, chrom2_flze)
    #mm = randrange(1,2)
    #print(mm)
    nn = randrange(1,smaller_fleet_size-1)
    #print("nn=")
    #print(nn)
    off_trip_raw = crossover(chrom1_trip, chrom2_trip, nn)
    off_flze_raw = len(off_trip_raw)
    
    #delete duplicate elements
    off_trip_fp = off_trip_raw[0:nn] #keep the first half part of the off_trip unchanged
    off_trip_sp = off_trip_raw[nn:off_flze_raw] #get the second half part of the off_trip
    off_trip_fp_com = list(itertools.chain.from_iterable(off_trip_fp)) #combine the elements in the first part of off_trip
    off_trip_sp_new = [[ele for ele in sub if ele not in off_trip_fp_com] for sub in off_trip_sp] #delete the elements in the second half part of the off_trip which are duplicated with the elements in the first half part
    off_trip_new = off_trip_fp + off_trip_sp_new #get the new off spring
    off_trip_new = list(filter(None, off_trip_new)) #delete empty trip chains in the new off spring
    
    #add missing elements
    off_trip_new_com = list(itertools.chain.from_iterable(off_trip_new)) #combine the elements in the new off spring
    all_trips = [*range(1,71,1)] #all service trips
    missing_trips = [ele for ele in all_trips if ele not in off_trip_new_com] #get the missing trips
    missing_trips.sort() #sort the missing trips and see it as a new trip chain
    new_trip_chain = missing_trips
    
    #get the trip part of the off spring
    off_trip_new.append(new_trip_chain) #append the new trip chain to the new off spring
    off_trip = off_trip_new
    #print("off_trip =")
    #print(off_trip)
    
    #get the fleet size of the off spring
    off_flze = len(off_trip)
    
    #get the battery size for each bus
    off_batt = battery_cap_generation(off_flze)
    battery_lower = off_batt[1]
    battery_upper = off_batt[2]
    
    #get the number of trips of each bus
    trips_per_bus = []
    for ele in off_trip:
        trips_per_bus.append(len(ele))
        
    #get whether to go back to the depot to charge between trips
    whether_to_charge = whether_charge_generation(off_flze, off_trip, trips_per_bus)
    
    #check whether the off spring is feasible
    check = check_feasibility(off_flze, trips_per_bus, battery_lower, battery_upper, off_trip, whether_to_charge, avail_char_time_between_trip_depot)
    #print(check)
    
    #store the offspring if it is feasible otherwise generate a new offspring
    while True:
        if check != 1:
            # generate the trip part of a new chromosome
            rest_trips = [i for i in range(1,71)]
            chromosome = []
            no_bus = 0
            no_trip = []
            trip_chains = []
            trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)
        
            off_trip=trippart_chromosome[0]
            off_flze=trippart_chromosome[1]
            trips_per_bus=trippart_chromosome[2]
        
            ### generate the battery capacity for each bus
            off_batt = battery_cap_generation(off_flze)
            battery_size = off_batt[0]
            battery_lower = off_batt[1]
            battery_upper = off_batt[2]
            
            #get whether to go back to the depot to charge between trips
            whether_to_charge = whether_charge_generation(off_flze, off_trip, trips_per_bus)
        
            ### check feasibility of the generated new chromosome
            check = check_feasibility(off_flze, trips_per_bus, battery_lower, battery_upper, off_trip, whether_to_charge, avail_char_time_between_trip_depot)
            #print(check)
            
        else:
            off_spring = {"fleet_size": off_flze, "trip_chromosome": off_trip, "battery_cap": off_batt, "whether_to_charge": whether_to_charge}
            break
            
    return off_spring

########################################
#mutation
########################################
def mutation(chrom):
    new_chrom = {}
    chrom_whether_to_charge = chrom["whether_to_charge"]
    chrom_trip = chrom["trip_chromosome"]
    chrom_batt = chrom["battery_cap"]
    chrom_flze = chrom["fleet_size"]
    
    trips_per_bus = []
    for ele in chrom_trip:
        trips_per_bus.append(len(ele))
    
    battery_lower = chrom_batt[1]
    battery_upper = chrom_batt[2]
    
    n = random. randint(1,2) 
    #print("n=")
    #print(n)
    
    if n == 1:
        jj = random.randrange(chrom_flze) 
        ss = random.randrange(trips_per_bus[jj])
        #print("jj=")
        #print(jj)
        chrom_whether_to_charge[jj][ss] = int(not chrom_whether_to_charge[jj][ss])
    elif n == 2:
        ii = random.sample(chrom_trip,2) #randomly choose two trip chains in chrom_trip
        #print("ii=")
        #print(ii)
        res_trip_chains = [ele for ele in chrom_trip if ele not in ii] #delete ii from chrom_trip
        #print("res_trip_chains=")
        #print(res_trip_chains)
        res_trip_com = list(itertools.chain.from_iterable(res_trip_chains))
        #print("res_trip_com=")
        #print(res_trip_com)
        ii_com = list(itertools.chain.from_iterable(ii)) #combine the elements in ii
        #print("ii_com")
        #print(ii_com)
        
        chromosome = []
        no_bus = 0
        no_trip = []
        trip_chains = []
        
        following_set_m = {key:[ele for ele in following_set[key] if ele not in res_trip_com] for key in following_set.keys()}
        #print("following_set_m=")
        #print(following_set_m)
        
        new_trip_chains = trippart_chromosome_generation(ii_com, chromosome, trip_chains, following_set_m, no_bus, no_trip)
        new_trip_chains = new_trip_chains[0]
        chrom_trip = res_trip_chains + new_trip_chains
        #print("chrom_trip=")
        #print(chrom_trip)
        chrom_flze = len(chrom_trip)
        #print("chrom_flze=")
        #print(chrom_flze)
        chrom_batt = battery_cap_generation(chrom_flze)
        
        battery_lower = chrom_batt[1]
        battery_upper = chrom_batt[2]
        
        #get the number of trips of each bus
        trips_per_bus = []
        for ele in chrom_trip:
            trips_per_bus.append(len(ele))
        
        #get whether to go back to the depot to charge between trips
        whether_to_charge = whether_charge_generation(chrom_flze, chrom_trip, trips_per_bus)
    
    #check feasibility of the new chromosome
    check = check_feasibility(chrom_flze, trips_per_bus, battery_lower, battery_upper, chrom_trip, whether_to_charge, avail_char_time_between_trip_depot)
    #print("check=")
    #print(check)
    
    #store the new chromosome if it is feasible otherwise generate a new chromosome
    while True:
        if check != 1:
            # generate the trip part of a new chromosome
            rest_trips = [i for i in range(1,71)]
            chromosome = []
            no_bus = 0
            no_trip = []
            trip_chains = []
            trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)
        
            chrom_trip=trippart_chromosome[0]
            chrom_flze=trippart_chromosome[1]
            trips_per_bus=trippart_chromosome[2]
        
            ### generate the battery capacity for each bus
            chrom_batt = battery_cap_generation(chrom_flze)
            battery_size = chrom_batt[0]
            battery_lower = chrom_batt[1]
            battery_upper = chrom_batt[2]
            
            #get whether to go back to the depot to charge between trips
            whether_to_charge = whether_charge_generation(chrom_flze, chrom_trip, trips_per_bus)
        
            ### check feasibility of the generated new chromosome
            check = check_feasibility(chrom_flze, trips_per_bus, battery_lower, battery_upper, chrom_trip, whether_to_charge, avail_char_time_between_trip_depot)
            #print("check=")
            #print(check)
        
        else:
            new_chrom = {"fleet_size": chrom_flze, "trip_chromosome": chrom_trip, "battery_cap": chrom_batt, "whether_to_charge": whether_to_charge}
            break
            
    return new_chrom     
    
######################################
#genetic algorithm
######################################

###########select
def select_mating_pool(init_population, num_parents):
    fitness_list = []
    fitness_return_list = []
    pop_size = len(init_population)
    for i in range(1,pop_size+1):
        chromosome = init_population[i]
        trip_chains = chromosome["trip_chromosome"]
        chrom_batt = chromosome["battery_cap"]
        fleet_size = chromosome["fleet_size"]
        whether_to_charge = chromosome["whether_to_charge"]
        print("whether_to_charge =")
        print(whether_to_charge)
        print(whether_to_charge[0][0])
        
        trips_per_bus = []
        for ele in trip_chains:
            trips_per_bus.append(len(ele))
            
        battery_lower = chrom_batt[1]
        battery_upper = chrom_batt[2]
            
        ########import the parameters of the chromosome to gams

        # store the time periods for charging of each bus to a dictionary
        dict_charging_time = {}
        
        for i in range(0, fleet_size):
            print("i=")
            print(i)
            trip_set = trip_chains[i]                   #get the trip chain of the bus i
            a = trips_per_bus[i]                        #get the number of trips for the bus i
            battery_l = battery_lower[i]                #get the battery capacity lower bound of the bus i
            battery_u = battery_upper[i]                #get the battery capacity upper bound of the bus i                
            b = trip_set[0]                               #get the first trip in the trip chain of the bus i
            remain_energy = battery_u - From_depot_to_trip_energy.iloc[b-1][1] - trip_information_3line.iloc[b-1]["Energy_need"]   #get the remaining energy in bus i after finishing its first trip
            if a == 1:
                remain_energy = remain_energy - From_trip_to_depot_energy.iloc[b-1][1]
                b_endtime = trip_information_3line.iloc[b-1]["End_time"]
                arrive_depot_time = b_endtime + math.ceil(From_trip_to_depot_time.iloc[b-1][1]*60)
                charging_time_need = math.ceil((battery_u - remain_energy)/350*60)
                for l in range(int(arrive_depot_time), int(arrive_depot_time +charging_time_need)):
                    dict_charging_time[str(i),str(l)] = 1
            else:
                j = 1
                while j < a:
                    print("j=")
                    print(j)
                    trip1 = trip_set[j-1]
                    trip2 = trip_set[j]
                    trip1_endtime = trip_information_3line.iloc[trip1-1]["End_time"]
                    trip2_endtime = trip_information_3line.iloc[trip2-1]["End_time"]
                    print(whether_to_charge)
                    whether_to_charge1 = whether_to_charge[i][j-1]
                    if j < a-1:
                        if whether_to_charge1 == 0:
                            remain_energy = remain_energy - energy_need.iloc[trip1-1][trip2] - trip_information_3line.iloc[trip2-1]["Energy_need"]
                        else:
                            remain_energy1 = remain_energy - From_trip_to_depot_energy.iloc[trip1-1][1]
                            arrive_depot_time = trip1_endtime + math.ceil(From_trip_to_depot_time.iloc[trip1-1][1]*60)
                            remain_energy2 = remain_energy1 + 350*avail_char_time_between_trip_depot.iloc[trip1-1][trip2-1]
                            if remain_energy2 >= battery_u:
                                remain_energy = battery_u
                                charging_time = math.ceil((battery_u - remain_energy1)/350*60)
                                for l in range(int(arrive_depot_time), int(arrive_depot_time +charging_time)):
                                    dict_charging_time[str(i),str(l)] = 1
                            else: 
                                remain_energy = remain_energy2
                                charging_time = math.ceil((remain_energy - remain_energy1)/350*60)
                                for l in range(int(arrive_depot_time), int(arrive_depot_time +charging_time)):
                                    dict_charging_time[str(i),str(l)] = 1
                            remain_energy = remain_energy - From_depot_to_trip_energy.iloc[trip2-1][1] - trip_information_3line.iloc[trip2-1]["Energy_need"]
                    else:
                        if whether_to_charge1 == 0:
                            remain_energy = remain_energy - energy_need.iloc[trip1-1][trip2] - trip_information_3line.iloc[trip2-1]["Energy_need"] - From_trip_to_depot_energy.iloc[trip2-1][1]
                            arrive_depot_time = trip2_endtime + math.ceil(From_trip_to_depot_time.iloc[trip2-1][1]*60)
                            charging_time_need = math.ceil((battery_u - remain_energy)/350*60)
                            for l in range(int(arrive_depot_time), int(arrive_depot_time +charging_time_need)):
                                dict_charging_time[str(i),str(l)] = 1
                        else:
                            remain_energy1 = remain_energy - From_trip_to_depot_energy.iloc[trip1-1][1]
                            arrive_depot_time = trip1_endtime + math.ceil(From_trip_to_depot_time.iloc[trip1-1][1]*60)
                            remain_energy2 = remain_energy1 + 350*avail_char_time_between_trip_depot.iloc[trip1-1][trip2-1]
                            if remain_energy2 > battery_u:
                                remain_energy = battery_u
                                charging_time = math.ceil((battery_u - remain_energy1)/350*60)
                                for l in range(int(arrive_depot_time), int(arrive_depot_time +charging_time)):
                                    dict_charging_time[str(i),str(l)] = 1
                            else: 
                                remain_energy = remain_energy2
                                charging_time = math.ceil((remain_energy - remain_energy1)/350*60)
                                for l in range(int(arrive_depot_time), int(arrive_depot_time +charging_time)):
                                    dict_charging_time[str(i),str(l)] = 1
                            remain_energy = remain_energy - From_depot_to_trip_energy.iloc[trip2-1][1] - trip_information_3line.iloc[trip2-1]["Energy_need"] - From_trip_to_depot_energy.iloc[trip2-1][1]
                            arrive_depot_time2 = trip2_endtime + math.ceil(From_trip_to_depot_time.iloc[trip2-1][1]*60)
                            charging_time2 = math.ceil((battery_u - remain_energy)/350*60)
                            for l in range(int(arrive_depot_time2), int(arrive_depot_time2 +charging_time2)):
                                dict_charging_time[str(i),str(l)] = 1
                    j += 1
        #print(dict_charging_time)

        # store the total operating time per day per bus
        dict_operating_time = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            first_trip = trip_set[0]
            last_trip = trip_set[-1]
            time1 = From_depot_to_trip_time.iloc[first_trip-1][1]
            first_trip_start_time = trip_information_3line.iloc[first_trip-1]["Start_time"]
            last_trip_end_time = trip_information_3line.iloc[last_trip-1]["End_time"]
            time2 = last_trip_end_time - first_trip_start_time
            time3 = From_trip_to_depot_time.iloc[last_trip-1][1]
            dict_operating_time[str(i)] = time1*60 + time2 + time3*60
        #print(dict_operating_time)  


        ws = GamsWorkspace(system_directory="/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS/", working_directory="/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS")
        # create new GamsDatabase instance
        db = ws.add_database()

        fleet_size_set = []
        i=1
        while i <= fleet_size:
            fleet_size_set.append(str(i))
            i += 1

        # add 1-dimensional set 'm' with explanatory text 'bus index' to the GamsDatabase
        m = db.add_set("m", 1, "bus index")
        for i in fleet_size_set:
            m.add_record(i)

        # add parameter 'batt' with domain 'm' with explanatory text 'battery capacity of bus bbb' to the GamsDatabase
        batt = db.add_parameter_dc("batt", [m], "battery capacity of bus bbb")
        for i in fleet_size_set:
            batt.add_record(i).value = chrom_batt[0][int(i)-1]

        # add parameter to define the charging time periods of each bus at the depot
        busct = GamsParameter(db, "busct", 2, "charging time periods of a bus at the depot")
        for k, v in dict_charging_time.items():
            busct.add_record(k).value = v

        # add parameter to define the operating time of each bus
        busst = GamsParameter(db, "busst", 1, "operating time of each bus")
        for k, v in dict_operating_time.items():
            busst.add_record(k).value = v

        # export the GamsDatabase to a GDX file with name 'data.gdx' located in the 'working_directory' of the GamsWorkspace
        db.export("/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS/data.gdx")

        # call and run gams model to calculate the fitness value of the chromosome
        gams_model = ws.add_job_from_file("EBus_scheduling_only_depot_charging.gms")
        gams_model.run()
        if (gams_model.out_db["modelStat"].find_record().value == 1 and gams_model.out_db["solveStat"].find_record().value == 1):
            for rec in gams_model.out_db["objvalue"]:
                fit_value = -rec.level
        else:
            fit_value = -999999999
        print(fit_value)
            
        fitness_list.append(fit_value)
        fitness_return_list.append(fit_value)

    fitness = np.array(fitness_list)
    fitness_return = np.array(fitness_return_list)
    #print(fitness_return)

    parents = {}
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num] = init_population[max_fitness_idx+1]
        fitness[max_fitness_idx] = -99999999999
    return parents,fitness_return 

###########crossover
def crossover_g(parents, offspring_size):
    num_parents = len(parents)
    offspring = {}
    for k in range(offspring_size):
        # Index of the first parent to mate.
        parent1_idx = k%num_parents
        # Index of the second parent to mate.
        parent2_idx = (k+1)%num_parents
        parent1 = parents[parent1_idx]
        parent2 = parents[parent2_idx]
        off_spring = cross_over(parent1, parent2)
        offspring[k] = off_spring
    return offspring

###########mutation
def mutation_g(offspring_crossover,mutationRate):
    num_mutation = int(mutationRate*len(offspring_crossover))
    for idx in range(num_mutation):
        mutation_idx = idx%num_mutation
        offspring_crossover[mutation_idx] = mutation(offspring_crossover[mutation_idx])
    return offspring_crossover

###########genetic algorithm
maximal_generation=150
population_size=120
parent_number=36
#maximal_generation=50
#population_size=20
#parent_number=16
mutation_rate=0.15
#number_of_variables = 3 #Battery size, charger number, charger power, charging threshold    
child_number = population_size - parent_number

In [ ]:
new_population = Initial_population
best_outputs = []
for i in range(maximal_generation):
    print("Generation: "+str(i))
    parents,fitness_return = select_mating_pool(new_population, parent_number)
    offspring_crossover = crossover_g(parents, offspring_size=child_number)
    offspring_mutation = mutation_g(offspring_crossover, mutation_rate)
    for key,value in parents.items():
        new_population[key+1] = value
    for key,value in offspring_mutation.items():
        new_population[key+parent_number+1] = value

    print(fitness_return)
    print(np.max(fitness_return))


    max_fitness_idx = np.where(fitness_return == np.max(fitness_return))
    max_fitness_idx = max_fitness_idx[0][0]
    best_solution = (np.max(fitness_return),new_population[max_fitness_idx+1])
    best_outputs.append(best_solution)

#print(best_solution)